# A1
* GCR = 0.25
* Albedo = 0.2
* Hub Height = 1.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* For a GCR of 0.25 the pitch is 9.536m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A1


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+245.gd0d9c92.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'A1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 9.536 # m
albedo = 0.2
hub_height = 1.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A1
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.25,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-01_0800', endtime='2022-02-01_0000')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~307 skyfiles. 
Created 304 skyfiles in /skies/


{'2022-01-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 55.0,
  'theta': -55.0,
  'dni': 0,
  'ghi': 78,
  'dhi': 76,
  'temp_air': 3.7,
  'wind_speed': 5.2,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_0830.rad'},
 '2022-01-01_0930': {'surf_azm': 90.0,
  'surf_tilt': 50.17,
  'theta': -50.17,
  'dni': 0,
  'ghi': 85,
  'dhi': 82,
  'temp_air': 3.3,
  'wind_speed': 5.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_0930.rad'},
 '2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1030.rad'},
 '2022-01-01_1130': {'surf_azm': 90.0,
  'surf_tilt': 4.27,
  'theta': -4.27,
  'dni': 60,
  'ghi': 265,
  'dhi': 232,
  'temp_air': 3.3,
  'wind_speed': 5.5,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1130.rad'},
 '2022-01-01_1230': {'surf_azm': 270.0,
  'surf_tilt': 21.33,
  'theta': 21.33,
  'dni': 0,
  'ghi': 121,
  'dhi': 118,

In [13]:
trackerdict = demo.makeScene1axis(module=module, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~304 .rad files for gendaylit 1-axis workflow (this takes a minute..)
304 Radfiles created in /objects/

Making 304 octfiles in root directory.
Created 1axis_2022-01-01_0830.oct
Created 1axis_2022-01-01_0930.oct
Created 1axis_2022-01-01_1030.oct
Created 1axis_2022-01-01_1130.oct
Created 1axis_2022-01-01_1230.oct
Created 1axis_2022-01-01_1330.oct
Created 1axis_2022-01-01_1430.oct
Created 1axis_2022-01-01_1530.oct
Created 1axis_2022-01-01_1630.oct
Created 1axis_2022-01-02_0830.oct
Created 1axis_2022-01-02_0930.oct
Created 1axis_2022-01-02_1030.oct
Created 1axis_2022-01-02_1130.oct
Created 1axis_2022-01-02_1230.oct
Created 1axis_2022-01-02_1330.oct
Created 1axis_2022-01-02_1430.oct
Created 1axis_2022-01-02_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.186245), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-02_1630.oct
Created 1axis_2022-01-03_0830.oct
Created 1axis_2022-01-03_0930.oct
Created 1axis_2022-01-03_1030.oct
Created 1axis_2022-01-03_1130.oct
Created 1axis_2022-01-03_1230.oct
Created 1axis_2022-01-03_1330.oct
Created 1axis_2022-01-03_1430.oct
Created 1axis_2022-01-03_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.117978), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-03_1630.oct
Created 1axis_2022-01-04_0730.oct
Created 1axis_2022-01-04_0830.oct
Created 1axis_2022-01-04_0930.oct
Created 1axis_2022-01-04_1030.oct
Created 1axis_2022-01-04_1130.oct
Created 1axis_2022-01-04_1230.oct
Created 1axis_2022-01-04_1330.oct
Created 1axis_2022-01-04_1430.oct
Created 1axis_2022-01-04_1530.oct


...

Created 1axis_2022-01-20_1630.oct
Created 1axis_2022-01-21_0730.oct
Created 1axis_2022-01-21_0830.oct
Created 1axis_2022-01-21_0930.oct
Created 1axis_2022-01-21_1030.oct
Created 1axis_2022-01-21_1130.oct
Created 1axis_2022-01-21_1230.oct
Created 1axis_2022-01-21_1330.oct
Created 1axis_2022-01-21_1430.oct
Created 1axis_2022-01-21_1530.oct
Created 1axis_2022-01-21_1630.oct
Created 1axis_2022-01-22_0730.oct
Created 1axis_2022-01-22_0830.oct
Created 1axis_2022-01-22_0930.oct
Created 1axis_2022-01-22_1030.oct
Created 1axis_2022-01-22_1130.oct
Created 1axis_2022-01-22_1230.oct
Created 1axis_2022-01-22_1330.oct
Created 1axis_2022-01-22_1430.oct
Created 1axis_2022-01-22_1530.oct
Created 1axis_2022-01-22_1630.oct
Created 1axis_2022-01-23_0730.oct
Created 1axis_2022-01-23_0830.oct
Created 1axis_2022-01-23_0930.oct
Created 1axis_2022-01-23_1030.oct
Created 1axis_2022-01-23_1130.oct
Created 1axis_2022-01-23_1230.oct
Created 1axis_2022-01-23_1330.oct
Created 1axis_2022-01-23_1430.oct
Created 1axis_

/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.014182), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-23_1630.oct
Created 1axis_2022-01-24_0730.oct
Created 1axis_2022-01-24_0830.oct
Created 1axis_2022-01-24_0930.oct
Created 1axis_2022-01-24_1030.oct
Created 1axis_2022-01-24_1130.oct
Created 1axis_2022-01-24_1230.oct
Created 1axis_2022-01-24_1330.oct
Created 1axis_2022-01-24_1430.oct
Created 1axis_2022-01-24_1530.oct
Created 1axis_2022-01-24_1630.oct
Created 1axis_2022-01-25_0730.oct
Created 1axis_2022-01-25_0830.oct
Created 1axis_2022-01-25_0930.oct
Created 1axis_2022-01-25_1030.oct
Created 1axis_2022-01-25_1130.oct
Created 1axis_2022-01-25_1230.oct
Created 1axis_2022-01-25_1330.oct
Created 1axis_2022-01-25_1430.oct
Created 1axis_2022-01-25_1530.oct
Created 1axis_2022-01-25_1630.oct
Created 1axis_2022-01-26_0730.oct
Created 1axis_2022-01-26_0830.oct
Created 1axis_2022-01-26_0930.oct
Created 1axis_2022-01-26_1030.oct
Created 1axis_2022-01-26_1130.oct
Created 1axis_2022-01-26_1230.oct
Created 1axis_2022-01-26_1330.oct
Created 1axis_2022-01-26_1430.oct
Created 1axis_

/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.019917), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-27_1630.oct
Created 1axis_2022-01-28_0730.oct
Created 1axis_2022-01-28_0830.oct
Created 1axis_2022-01-28_0930.oct
Created 1axis_2022-01-28_1030.oct
Created 1axis_2022-01-28_1130.oct
Created 1axis_2022-01-28_1230.oct
Created 1axis_2022-01-28_1330.oct
Created 1axis_2022-01-28_1430.oct
Created 1axis_2022-01-28_1530.oct
Created 1axis_2022-01-28_1630.oct
Created 1axis_2022-01-29_0730.oct
Created 1axis_2022-01-29_0830.oct
Created 1axis_2022-01-29_0930.oct
Created 1axis_2022-01-29_1030.oct
Created 1axis_2022-01-29_1130.oct
Created 1axis_2022-01-29_1230.oct
Created 1axis_2022-01-29_1330.oct
Created 1axis_2022-01-29_1430.oct
Created 1axis_2022-01-29_1530.oct
Created 1axis_2022-01-29_1630.oct
Created 1axis_2022-01-30_0730.oct
Created 1axis_2022-01-30_0830.oct
Created 1axis_2022-01-30_0930.oct
Created 1axis_2022-01-30_1030.oct
Created 1axis_2022-01-30_1130.oct
Created 1axis_2022-01-30_1230.oct
Created 1axis_2022-01-30_1330.oct
Created 1axis_2022-01-30_1430.oct
Created 1axis_

In [14]:
#trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

In [15]:
trackerdict

{'2022-01-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 55.0,
  'theta': -55.0,
  'dni': 0,
  'ghi': 78,
  'dhi': 76,
  'temp_air': 3.7,
  'wind_speed': 5.2,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_0830.rad',
  'radfile': 'objects/1axis2022-01-01_0830__C_0.60139_rtr_9.53600_tilt_55.00000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1.303, 'y': 2.384, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.305, 'sceney': 2.384, 'scenez': 0.095, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1.303 2.384 0.02 | xform -t -0.6515 -1.192 0.095 -a 1 -t 0 2.384 0\r\n! genrev Metal_Grey tube1 t*1.305 0.075 32 | xform -ry 90 -t -0.6525 0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0.095, 'xgap': 0.002, 'ygap': 0.0, 'zgap': 0.02}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.25, 'text': '!xform -rx 55.0 -t 0 0 1.5 -a 25 -t 1.305 0 0 -a 5 -t 0 9.536 0 -i 1 -t -15.66 -19.072 0 -rz 90.0 -t 0 0 0 objects/PVmod.rad', '

In [16]:
demo

{'metdata': {'latitude': 35.05, 'longitude': -106.54, 'elevation': 1600.0, 'timezone': -7.0, 'city': 'Albuquerque', 'datetime': [Timestamp('2022-01-01 08:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 09:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 10:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 11:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 12:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 13:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 14:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 15:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 16:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-02 08:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-02 09:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-02 10:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-02 11:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-02 12:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-02 13:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-02 14:30:00-0700', tz='Etc/

In [17]:
#trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

In [18]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

Linescan in process: 1axis_2022-01-01_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_0830_Row3_Module25.csv
Index: 2022-01-01_0830. Wm2Front: 61.21388833333333. Wm2Back: 18.752048333333335
Linescan in process: 1axis_2022-01-01_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_0930_Row3_Module25.csv
Index: 2022-01-01_0930. Wm2Front: 66.32928666666666. Wm2Back: 18.817261666666667
Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module25.csv
Index: 2022-01-01_1030. Wm2Front: 102.84875. Wm2Back: 17.479893333333333
Linescan in process: 1axis_2022-01-01_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_1130_Row3_Module25.csv
Index: 2022-01-01_1130. Wm2Fr

Saved: results/irr_1axis_2022-01-04_1030_Row3_Module25.csv
Index: 2022-01-04_1030. Wm2Front: 620.5853166666667. Wm2Back: 44.048944999999996
Linescan in process: 1axis_2022-01-04_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-01-04_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-04_1130_Row3_Module25.csv
Index: 2022-01-04_1130. Wm2Front: 589.2135666666666. Wm2Back: 50.32312666666667
Linescan in process: 1axis_2022-01-04_1230_Row3_Module25_Front
Linescan in process: 1axis_2022-01-04_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-04_1230_Row3_Module25.csv
Index: 2022-01-04_1230. Wm2Front: 611.4305333333334. Wm2Back: 48.041425
Linescan in process: 1axis_2022-01-04_1330_Row3_Module25_Front
Linescan in process: 1axis_2022-01-04_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-04_1330_Row3_Module25.csv
Index: 2022-01-04_1330. Wm2Front: 674.7707833333334. Wm2Back: 39.77422
Linescan in process: 1axis_2022-01-04_1430_Row3_Module25_Front
Linescan in process: 1

Linescan in process: 1axis_2022-01-07_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-07_1130_Row3_Module25.csv
Index: 2022-01-07_1130. Wm2Front: 581.24665. Wm2Back: 49.081446666666665
Linescan in process: 1axis_2022-01-07_1230_Row3_Module25_Front
Linescan in process: 1axis_2022-01-07_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-07_1230_Row3_Module25.csv
Index: 2022-01-07_1230. Wm2Front: 603.1531166666666. Wm2Back: 47.852455
Linescan in process: 1axis_2022-01-07_1330_Row3_Module25_Front
Linescan in process: 1axis_2022-01-07_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-07_1330_Row3_Module25.csv
Index: 2022-01-07_1330. Wm2Front: 662.5685833333333. Wm2Back: 39.04749666666666
Linescan in process: 1axis_2022-01-07_1430_Row3_Module25_Front
Linescan in process: 1axis_2022-01-07_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-07_1430_Row3_Module25.csv
Index: 2022-01-07_1430. Wm2Front: 715.3384333333333. Wm2Back: 31.48785
Linescan in process: 1axis_2022

Saved: results/irr_1axis_2022-01-10_1130_Row3_Module25.csv
Index: 2022-01-10_1130. Wm2Front: 604.21535. Wm2Back: 51.42393833333333
Linescan in process: 1axis_2022-01-10_1230_Row3_Module25_Front
Linescan in process: 1axis_2022-01-10_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-10_1230_Row3_Module25.csv
Index: 2022-01-10_1230. Wm2Front: 624.2544. Wm2Back: 49.55488166666666
Linescan in process: 1axis_2022-01-10_1330_Row3_Module25_Front
Linescan in process: 1axis_2022-01-10_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-10_1330_Row3_Module25.csv
Index: 2022-01-10_1330. Wm2Front: 683.0949. Wm2Back: 41.836618333333334
Linescan in process: 1axis_2022-01-10_1430_Row3_Module25_Front
Linescan in process: 1axis_2022-01-10_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-10_1430_Row3_Module25.csv
Index: 2022-01-10_1430. Wm2Front: 731.4370833333334. Wm2Back: 34.76560333333333
Linescan in process: 1axis_2022-01-10_1530_Row3_Module25_Front
Linescan in process: 1axis_2022

Saved: results/irr_1axis_2022-01-13_1130_Row3_Module25.csv
Index: 2022-01-13_1130. Wm2Front: 599.8188. Wm2Back: 52.54629666666666
Linescan in process: 1axis_2022-01-13_1230_Row3_Module25_Front
Linescan in process: 1axis_2022-01-13_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-13_1230_Row3_Module25.csv
Index: 2022-01-13_1230. Wm2Front: 627.5374499999999. Wm2Back: 50.58466166666666
Linescan in process: 1axis_2022-01-13_1330_Row3_Module25_Front
Linescan in process: 1axis_2022-01-13_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-13_1330_Row3_Module25.csv
Index: 2022-01-13_1330. Wm2Front: 688.7006666666666. Wm2Back: 41.9182
Linescan in process: 1axis_2022-01-13_1430_Row3_Module25_Front
Linescan in process: 1axis_2022-01-13_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-13_1430_Row3_Module25.csv
Index: 2022-01-13_1430. Wm2Front: 742.08995. Wm2Back: 34.32167
Linescan in process: 1axis_2022-01-13_1530_Row3_Module25_Front
Linescan in process: 1axis_2022-01-13_1530

Linescan in process: 1axis_2022-01-16_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-16_1230_Row3_Module25.csv
Index: 2022-01-16_1230. Wm2Front: 652.7482833333333. Wm2Back: 52.95277833333334
Linescan in process: 1axis_2022-01-16_1330_Row3_Module25_Front
Linescan in process: 1axis_2022-01-16_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-16_1330_Row3_Module25.csv
Index: 2022-01-16_1330. Wm2Front: 702.5424. Wm2Back: 45.07216166666666
Linescan in process: 1axis_2022-01-16_1430_Row3_Module25_Front
Linescan in process: 1axis_2022-01-16_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-16_1430_Row3_Module25.csv
Index: 2022-01-16_1430. Wm2Front: 771.1067333333333. Wm2Back: 35.335701666666665
Linescan in process: 1axis_2022-01-16_1530_Row3_Module25_Front
Linescan in process: 1axis_2022-01-16_1530_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-16_1530_Row3_Module25.csv
Index: 2022-01-16_1530. Wm2Front: 690.7205333333334. Wm2Back: 27.02981666666667
Linescan in pro

Linescan in process: 1axis_2022-01-19_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-19_1330_Row3_Module25.csv
Index: 2022-01-19_1330. Wm2Front: 203.35323333333332. Wm2Back: 38.996986666666665
Linescan in process: 1axis_2022-01-19_1430_Row3_Module25_Front
Linescan in process: 1axis_2022-01-19_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-19_1430_Row3_Module25.csv
Index: 2022-01-19_1430. Wm2Front: 259.3929166666667. Wm2Back: 54.438224999999996
Linescan in process: 1axis_2022-01-19_1530_Row3_Module25_Front
Linescan in process: 1axis_2022-01-19_1530_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-19_1530_Row3_Module25.csv
Index: 2022-01-19_1530. Wm2Front: 156.25206666666668. Wm2Back: 33.263415
Linescan in process: 1axis_2022-01-19_1630_Row3_Module25_Front
Linescan in process: 1axis_2022-01-19_1630_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-19_1630_Row3_Module25.csv
Index: 2022-01-19_1630. Wm2Front: 82.72780333333334. Wm2Back: 11.15838
Linescan in process:

Linescan in process: 1axis_2022-01-22_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-22_1430_Row3_Module25.csv
Index: 2022-01-22_1430. Wm2Front: 601.6816333333334. Wm2Back: 42.8441
Linescan in process: 1axis_2022-01-22_1530_Row3_Module25_Front
Linescan in process: 1axis_2022-01-22_1530_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-22_1530_Row3_Module25.csv
Index: 2022-01-22_1530. Wm2Front: 641.7294333333333. Wm2Back: 34.08182333333333
Linescan in process: 1axis_2022-01-22_1630_Row3_Module25_Front
Linescan in process: 1axis_2022-01-22_1630_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-22_1630_Row3_Module25.csv
Index: 2022-01-22_1630. Wm2Front: 121.58405. Wm2Back: 6.777209
Linescan in process: 1axis_2022-01-23_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-01-23_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-23_0730_Row3_Module25.csv
Index: 2022-01-23_0730. Wm2Front: 354.86458333333337. Wm2Back: 7.412559833333334
Linescan in process: 1axis_2022-0

Linescan in process: 1axis_2022-01-25_1530_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-25_1530_Row3_Module25.csv
Index: 2022-01-25_1530. Wm2Front: 757.2345. Wm2Back: 31.29682833333333
Linescan in process: 1axis_2022-01-25_1630_Row3_Module25_Front
Linescan in process: 1axis_2022-01-25_1630_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-25_1630_Row3_Module25.csv
Index: 2022-01-25_1630. Wm2Front: 249.46438333333333. Wm2Back: 3.2474311666666664
Linescan in process: 1axis_2022-01-26_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-01-26_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-26_0730_Row3_Module25.csv
Index: 2022-01-26_0730. Wm2Front: 94.36610666666667. Wm2Back: 11.990965
Linescan in process: 1axis_2022-01-26_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-01-26_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-26_0830_Row3_Module25.csv
Index: 2022-01-26_0830. Wm2Front: 172.16469999999998. Wm2Back: 30.422428333333333
Linescan in process:

Linescan in process: 1axis_2022-01-28_1630_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-28_1630_Row3_Module25.csv
Index: 2022-01-28_1630. Wm2Front: 290.7554. Wm2Back: 3.4000255
Linescan in process: 1axis_2022-01-29_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-01-29_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-29_0730_Row3_Module25.csv
Index: 2022-01-29_0730. Wm2Front: 454.9358166666667. Wm2Back: 7.2433285000000005
Linescan in process: 1axis_2022-01-29_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-01-29_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-29_0830_Row3_Module25.csv
Index: 2022-01-29_0830. Wm2Front: 713.7359166666666. Wm2Back: 31.530068333333332
Linescan in process: 1axis_2022-01-29_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-01-29_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-29_0930_Row3_Module25.csv
Index: 2022-01-29_0930. Wm2Front: 758.9006333333334. Wm2Back: 39.621941666666665
Linescan in process: 

In [19]:
demo.calculateResults()

Bifaciality factor of module stored is  1
No CECModule data passed; using default for Prism Solar BHC72-400


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

{'2022-01-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 55.0,
  'theta': -55.0,
  'dni': 0,
  'ghi': 78,
  'dhi': 76,
  'temp_air': 3.7,
  'wind_speed': 5.2,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_0830.rad',
  'radfile': 'objects/1axis2022-01-01_0830__C_0.60139_rtr_9.53600_tilt_55.00000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1.303, 'y': 2.384, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.305, 'sceney': 2.384, 'scenez': 0.095, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1.303 2.384 0.02 | xform -t -0.6515 -1.192 0.095 -a 1 -t 0 2.384 0\r\n! genrev Metal_Grey tube1 t*1.305 0.075 32 | xform -ry 90 -t -0.6525 0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0.095, 'xgap': 0.002, 'ygap': 0.0, 'zgap': 0.02}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.25, 'text': '!xform -rx 55.0 -t 0 0 1.5 -a 25 -t 1.305 0 0 -a 5 -t 0 9.536 0 -i 1 -t -15.66 -19.072 0 -rz 90.0 -t 0 0 0 objects/PVmod.rad', '

In [20]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

,module,row,POA_eff,Grear_mean,Gfront_mean,Module_temp,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
timestamp,,,,,,,,,,,,,,,,
2022-01-01_0830,25,3,79.965937,18.752048,61.213888,5.479618,32.942182,25.016529,30.633650,31.681664,0.001315,32.941749,5.2,0,76,78
2022-01-01_0930,25,3,85.146548,18.817262,66.329287,5.219688,35.188293,27.214675,28.369462,29.298967,0.001425,35.187792,5.0,0,82,85
2022-01-01_1030,25,3,120.328643,17.479893,102.848750,6.666411,50.033290,42.609227,16.995728,17.423603,0.000964,50.032807,4.7,5,108,112
2022-01-01_1130,25,3,294.856778,32.575978,262.280800,9.735635,124.223093,110.502115,12.420268,12.416938,0.000022,124.223066,5.5,60,232,265
2022-01-01_1230,25,3,130.487313,17.566630,112.920683,5.239960,54.686437,47.172390,15.556610,15.928908,0.000533,54.686146,6.1,0,118,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31_0930,25,3,809.661485,40.375802,769.285683,26.651056,323.226542,308.567958,5.248480,4.750520,0.000173,323.225984,1.6,991,36,444
2022-01-31_1030,25,3,782.926277,52.589343,730.336933,27.032714,312.240559,292.924382,7.200696,6.594254,0.000118,312.240191,2.7,1040,43,594
2022-01-31_1130,25,3,758.277305,59.756555,698.520750,27.952443,301.484884,279.510386,8.554729,7.861782,0.000011,301.484852,2.5,1063,45,683


In [21]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A1.csv')